In [18]:
import wandb
from math import isnan 
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import wandb

api = wandb.Api()
projects= ["Blines", "LayerSelection"]
# Project is specified by <entity/project-name>
summary_list = [] 
config_list = [] 
name_list = [] 
id_list = []
algo_list = []
for project in projects:
  runs = api.runs(f"alelab/{project}")
  for run in runs: 
    # run.summary are the output key/values like accuracy.
    # We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict) 

    # run.config is the input metrics.
    # We remove special values that start with _.
    config = {k:v for k,v in run.config.items() if not k.startswith('_')}
    config["bit_width_list"] = list(map(int, config["bit_width_list"].split(',')))
    config["bit_width"] = config["bit_width_list"][0]
    config_list.append(config) 

    # run.name is the name of the run.
    name_list.append(run.name)
    id_list.append(run.id)
    if "HIGH" in run.name:
        algo_list.append("OURS(high)")
    if "LOW" in run.name:
        algo_list.append("OURS(low)")
    elif "vanilla" in run.name:
        algo_list.append("Vanilla QAT")
    elif "anyprec" in run.name:
        algo_list.append("Anyprecision")
   
summary_df = pd.DataFrame.from_records(summary_list) 
config_df = pd.DataFrame.from_records(config_list) 
name_df = pd.DataFrame({'name': name_list}) 
id_df = pd.DataFrame({'id': id_list})
algo_df = pd.DataFrame({'algorithm': algo_list})
all_df = pd.concat([name_df, config_df,summary_df,algo_df, id_df], axis=1)

In [20]:
all_df.columns

Index(['name', 'lr', 'seed', 'model', 'epochs', 'resume', 'dataset', 'project',
       'workers', 'lr_decay',
       ...
       'dual_layer_1_bw_2', 'slack_layer_6_bw_2_train',
       'slack_layer_8_bw_2_train', 'dual_layer_12_bw_2', 'slack_CE_bw_2_train',
       'slack_layer_3_bw_2_test', 'slack_layer_1_bw_2_train',
       'dual_layer_2_bw_2', 'algorithm', 'id'],
      dtype='object', length=269)

In [19]:
for algo in ["OURS(high)", "OURS(low)", "Vanilla QAT", "Anyprecision"]:
    for bw in [2,]:
        query_df = all_df.query(f'bit_width=={bw}')
        for layer in range(11):
            
        print(query_df.head())

                    name     lr  seed      model  epochs resume  dataset  \
35  HIGH_bw_2_eps_0.0555  0.001     2  resnet20q     100   None  cifar10   
39  HIGH_bw_2_eps_0.0555  0.001     1  resnet20q     100   None  cifar10   
45  HIGH_bw_2_eps_0.0555  0.001     0  resnet20q     100   None  cifar10   

           project  workers   lr_decay  ... dual_layer_1_bw_2  \
35  LayerSelection        0  50,75,100  ...          0.001646   
39  LayerSelection        0  50,75,100  ...          0.000475   
45  LayerSelection        0  50,75,100  ...          0.004841   

   slack_layer_6_bw_2_train  slack_layer_8_bw_2_train  dual_layer_12_bw_2  \
35                 0.050774                  0.046345            1.098563   
39                 0.046540                  0.048901            1.094828   
45                 0.051899                  0.047304            1.094365   

    slack_CE_bw_2_train slack_layer_3_bw_2_test  slack_layer_1_bw_2_train  \
35            -0.006842                0.019886 